In [15]:
import pytest
import numpy as np
import aspect_based_sentiment_analysis as absa
from aspect_based_sentiment_analysis.training import ConfusionMatrix


@pytest.fixture
def nlp() -> absa.Pipeline:
    sentencier = absa.sentencizer()
    recognizer = absa.probing.AttentionGradientProduct()
    nlp = absa.load(text_splitter=sentencier, pattern_recognizer=recognizer)
    return nlp


@pytest.mark.slow
@pytest.mark.timeout(25)  # First 10s requires a pipeline to initialize.
def test_inference(nlp: absa.Pipeline):
    text = ("We are great fans of Slack, but we wish the subscriptions "
        "were more accessible to small startups.")
    aspects = ["slack","price"]
    return nlp(text, aspects)


@pytest.mark.slow
def test_semeval_classification_restaurants():
    examples = absa.load_examples(dataset='semeval',
                                  domain='laptop',
                                  test=True)
#     nlp = absa.load('absa/bert-lapt-0.1')

    metric = ConfusionMatrix(num_classes=3)
    confusion_matrix = nlp.evaluate(examples, metric, batch_size=32)
    confusion_matrix = confusion_matrix.numpy()
    accuracy = np.diagonal(confusion_matrix).sum() / confusion_matrix.sum()
    assert round(accuracy, 3) >= 0.86
    
    print("Accuracy:", accuracy)
    return confusion_matrix


In [16]:
nlp = absa.load('absa/bert-lapt-0.1')

results = test_inference(nlp)

Some weights of the model checkpoint at absa/bert-lapt-0.1 were not used when initializing BertABSClassifier: ['dropout_37', 'extractor']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertABSClassifier were not initialized from the model checkpoint at absa/bert-lapt-0.1 and are newly initialized: ['dropout_113']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
for aspect in results.aspects:
    print("\n Aspect: ", aspect)
    print("Sentiment: ",results.subtasks[aspect].examples[0].sentiment)


 Aspect:  slack
Sentiment:  Sentiment.negative

 Aspect:  price
Sentiment:  Sentiment.negative
